In [ ]:
#Scraper for BPHN
# 1. Peraturan DPRD
# 2. Peraturan Bersama
# 3. MOU
# 4. PKS
# 5. Peraturan PT
# 6. Keputusan PT
# 7. SE PT
# 8. Kerjasama PT
# 9. Intl Agreement
# 10. Kolonial
# 11. Monografi
# 12. Artikel

In [ ]:
import pandas as pd

df = pd.read_json('data_jdihn.json')

In [ ]:
df

In [ ]:
import requests
import json
from bs4 import BeautifulSoup as bs

In [ ]:

import numpy as np
import math
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import urllib3

In [ ]:
class peraturan: 
    def __init__(self, source, jenis, judul, link): 
        self.source = source 
        self.jenis = jenis
        self.judul = judul 
        self.link = link

In [ ]:
def extractPage(s,headers,jenis,token,page):
    payload = {}
    payload['jenis'] = jenis
    payload['_token'] = token
    payload['page'] = page
    print(payload)
    try:
        r = s.post('https://jdihn.go.id/pencarian', headers=headers, params=payload)
        soup = bs(r.content, 'html.parser')
        
        data_peraturan = []
        lists = soup.find_all('div',{'class':'row align-items-md-center align-items-lg-stretch'})
        for list in lists:
            div_source = list.find("a",{'class': 'media-title'})
            div_jenis = list.find("span",{'class': 'content-view'})
            div_judul = list.find("a",{'class': 'media-title text-capitalize'})
            div_links = list.findAll("a")
            print(div_judul.text)
            data_peraturan.append(peraturan(div_source.text,div_jenis.text,div_judul.text,div_links[2]["href"]))
        return data_peraturan
    except urllib3.exceptions.ProtocolError:
        print("error!!")
        return

In [ ]:


headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}
page_size = 8

with requests.Session() as s:
    r = s.get('https://jdihn.go.id/dokumen-hukum', headers=headers)
    
    soup = bs(r.content, 'html.parser')
    token = soup.find('input',{'name':'_token'})
    
    data_peraturan_all = []
    with ThreadPoolExecutor(max_workers=20) as exe:
                 
        for index, row in df.iterrows():
            page_total = int(row["jumlah"])

            est_page = math.ceil(page_total/page_size)
            print(row["name"])
            pages = np.arange(1, 1+est_page , 1)
            print(pages)

            futures = [exe.submit(extractPage, s, r.headers, row["name"], token["value"], page) for page in pages]       

            for future in as_completed(futures):
               data_peraturan_all.append(future.result())


In [ ]:
len(data_peraturan_all)

In [ ]:
import itertools
data_peraturan_bphn= list(itertools.chain.from_iterable(data_peraturan_all))

In [ ]:
len(data_peraturan_all)

In [ ]:
data_peraturan_bphn[0].link

In [ ]:
import pandas as pd

In [ ]:
lst = [[x.source, x.jenis, x.judul, x.link] for x in data_peraturan_bphn]

In [ ]:
df = pd.DataFrame(lst,columns=['source','jenis','judul','link'])
df.to_csv('bphn.csv')